# <b>Case Study Overview</b>
Lending Club is a consumer finance company specialized in lending various types of loans. Upon receiving a loan application, the company has to make a decision for loan approval based on the applicant’s profile. 

Applicants who don't repay the loan, i.e., defaulted on the loan, cause the largest amount of loss to the lenders. If one is able to identify these risky loan applicants, then such loans can be reduced thereby cutting down the amount of financial loss.

# <b>Objective</b>
The company wants to understand the driving factors behind loan default, i.e. they want to identify the variables which are strong indicators of default. 
They can then use those variables, at the time of loan application, for approval/rejection of the loan.

---
# Step 1. Import the necessary libraries
Before loading the dataset, you will need to first import all the relevant libraries. If you don’t have them installed already, you can do so by using the pip install command.<br>
Import pandas, numpy and matplolib.

In [1]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
```
</details>

# Step 2. Loading the data
You can now load the dataset into pandas  using the `read_csv()` function. This converts the CSV file into a Pandas DataFrame.<br>
You can set `dtype` parameter to 'O' to specify that all columns have data type <i>object</i>. This is not necessary but will ensure there are no columns with mixed types and will suppress a DtypeWarning.

In [2]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df = pd.read_csv("loan.csv", dtype='O')
```
</details>

# Step 3. Viewing the data

The dataset contains information about past issued loans and whether they were 'defaulted' or not. 

You can check the "Data_Dictionary.xlsx" file to see the description of each column.

## See the first 10 entries

In [3]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df.head(10)
```
</details>

## How many loans/rows are in the data? How many features/columns are in the data?

In [4]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>

There are 39717 rows and 111 columns.

```python
loan_df.shape
```
</details>

## Does the data have any members with more than one loan application?

In [5]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use `member_id` column
- Check if there is <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.any.html"><code>any</code></a> <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.duplicated.html"><code>duplicated</code></a> rows for that column.
</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

No members in the data are taking more than one loan.

```python
loan_df["member_id"].duplicated().any()
```
</details>

# Step 4. Preparing the data

We need to do some cleaning and feature transformation to get the data ready for the main EDA task.

# Step 4.1. Remove Irrelevant Columns    

There are broadly three types of features: 
- Customer profile (employment details, income etc.)
- Loan characteristics (amount of loan, interest rate, purpose of loan etc.)
- Customer behaviour variables (those which are generated after the loan is approved such as delinquent 2 years, revolving balance, next payment date etc.).

The customer behaviour variables are not available at the time of loan application, and thus they cannot be used as predictors for credit approval.

## Remove customer behaviour columns

In [6]:
behaviour_columns = ['last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'delinq_2yrs', 
                     'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
                    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
                    'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv',
                    'recoveries', 'collection_recovery_fee']

In [7]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html" target="_blank"><code>drop()</code></a> method
- Don't forget to reassign the DataFrame or use inplace = True
</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df = loan_df.drop(behaviour_columns, axis=1)
```

Note: 20 columns dropped (111 -> 91)
</details>

## Remove columns with constant value

Remove columns that, for every row, have the same constant value, since they will not give us any insights.

In [8]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- For each column, the number of unique values (see <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html"><code>nunique()</code></a>) needs to be greater than 1
- Use <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html"><code>loc[]</code></a> to maintain all rows but only keep the columns that satisfy the condition
- Don't forget to reassign the DataFrame

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df = loan_df.loc[:, loan_df.nunique() > 1]
```

Note: 63 columns dropped (91 -> 28)
</details>

## Remove columns with distinct values for all rows

Remove all columns that have different values for each row, since they will not give us any insights.

In [9]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- For each column, the number of unique values (see <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html"><code>nunique()</code></a>) needs to be less than the total number of rows
- Use <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html"><code>loc[]</code></a> to maintain all rows but only keep columns that satisfy the condition.
- Don't forget to reassign the DataFrame

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df = loan_df.loc[:, loan_df.nunique() < loan_df.shape[0]]
```

Note: 3 columns dropped (28 -> 25)
</details>

# Step 4.2. Handle Missing values

## Calculate the percentage of missing values for each column

In [10]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html"><code>isnull()</code></a> and <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html"><code>sum()</code></a> to get the frequency of missing columns for each column
- Divide by total number of rows to get the relative frequency 

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df.isnull().sum() / loan_df.shape[0] *100
```
</details>

## Remove columns that have more than 50% missing values

In [11]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Create a condition based on the answer from the previous exercise
- Use <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html"><code>loc[]</code></a> to maintain all rows but only keep the columns that satisfy the condition
- Don't forget to reassign the DataFrame

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
null_perc = loan_df.isnull().sum() / loan_df.shape[0] *100
loan_df = loan_df.loc[:, null_perc <= 50]
```

Note: 3 columns dropped (25 -> 22)
</details>

We have 4 columns left with missing values:
1. "title" with only 11 nulls
2. "pub_rec_bankruptcies" with 1.75% of nulls
3. "emp_title" with 6.19% of nulls
4. "emp_length" with 2.71% of nulls

## Remove <i>title</i> and <i>desc</i> columns
We can remove <i>title</i> and <i>desc</i> columns since <i>purpose</i> is a more useful column with the reason provided by the customer for the loan request. 

In [12]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html" target="_blank"><code>drop()</code></a> method
- Don't forget to reassign the DataFrame or use inplace = True
</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df = loan_df.drop(["title", "desc"], axis=1)
```

</details>

## Remove rows where either <i>pub_rec_bankruptcies</i> or <i>emp_length</i> have missing values

Since both columns have a low percentage of missing values and we don't want to introduce unnecessary bias, it is safer to remove all rows with null values for each of these columns.

In [13]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html" target="_blank"><code>dropna()</code></a> method
- Don't forget to reassign the DataFrame or use inplace = True
</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df = loan_df.dropna(subset=['pub_rec_bankruptcies', 'emp_length'])
```

</details>

## How many unique values does column <i>emp_title</i> have?

Do you think this column is meaningful for our analysis? If not, remove the column; otherwise, what approach would you use to deal with the missing values?

In [14]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>

There are 28299 unique values in column "emp_title", which is almost 75% of the total number of records.

```python
loan_df['emp_title'].nunique() #/ loan_df.shape[0] *100
```

We can assume this column is not meaningful for our analysis and remove the column:
```python
loan_df = loan_df.drop("emp_title", axis=1)
```

</details>

# Step 4.3. Remove Duplicates

## Does the data have duplicated rows?

If so, remove all duplicates.

In [15]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html"><code>duplicated()</code></a> and <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html"><code>any()</code></a>. 
</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>
There are no duplicate records in our data.

```python
loan_df.duplicated().any()
```

</details>

# Step 4.4. Change data formats and transform text data

### Remove text data from <i>term</i> column and convert to int datatype 

Remove the word "months" and change the datatype

In [16]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html"><code>str.split()</code></a> and <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get.html"><code>str.get()</code></a> to extract only the number from <i>term</i>
- To convert to integer use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html"><code>astype()</code></a>
- Don't forget to reassign the column Series

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df['term'] = loan_df['term'].str.split().str.get(0).astype(int)

#or
loan_df['term'].str.replace(" months", "").astype(int)
```

</details>

### Remove "%" symbol from <i>int_rate</i> and change the datatype to float

In [17]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.rstrip.html"><code>str.rstrip()</code></a> to strip specified characters from each string in the Series from right side
- To convert to float use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html"><code>astype()</code></a>
- Don't forget to reassign the column Series

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df['int_rate'] = loan_df['int_rate'].str.rstrip('%').astype(float)

#or
loan_df["term"].str.replace("%","").astype(float)
```

</details>

### Convert <i>emp_length</i> values to numeric

- "< 1 year" should be replaced by 0
- "10+ years" replaced by 10
- The rest should be replaced by their magnitude - e.g., "6 years" to 6

In [18]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

- You can use <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html" target="_blank"><code>replace()</code></a> by passing a dict where each key is an existing value and each value is the respective replacement value.<br>
Alternatively, you can make small changes one at a time using chained <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html" target="_blank"><code>str.replace()</code></a>.
- Don't forget to reassign the column Series

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
replace_dict = { 
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}
loan_df['emp_length'] = loan_df['emp_length'].replace(replace_dict)
```

</details>

### Create bins for <i>int_rate</i> and <i>annual_inc</i>

To go from a continuous feature, like <i>int_rate</i>, to a categorical feature, we can group its values into different bins. 

We can use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html" target="_blank"><code>pd.cut()</code></a> to perform this computation as below:

In [19]:
### Uncomment and run this code cell ###
# bins = [0, 7.5, 10, 12.5, 15, np.inf]
# labels = ['0-7.5', '7.5-10', '10-12.5', '12.5-15', '15+']

# loan_df['int_rate_range'] = pd.cut(loan_df['int_rate'], bins=bins, labels=labels)

# loan_df[['int_rate', 'int_rate_range']].head()

For the column <i>annual_inc</i>, first make sure its data type is float. Then, bin values into discrete intervals as follows:
- People earning less than USD 40,000
- People earning between USD 40,000 to USD 70,000
- People earning between USD 70,000 to USD 100,000
- People earning more than USD 100,000

Assign the result to a new column <i>annual_inc_range</i>.

In [20]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df['annual_inc'] = loan_df['annual_inc'].astype(float)

bins = np.array([0, 40, 70, 100, np.inf]) * 1000 
labels = ['$0-$40K', '$40K-$70K', '$70K-$100K', '$100K+']

loan_df['annual_inc_range'] = pd.cut(loan_df['annual_inc'], bins, labels=labels)
```

</details>

# Step 4.5. Target variable processing

The feature that tells us if a loan was paid or defaulted is <i>loan_status</i>.

We want to compare this variable across the other variables and identify the ones that affect it the most.

### What is the datatype and unique values of the column <i>loan_status</i>?

In [21]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df['loan_status'].unique()

loan_df['loan_status'].dtype
```

</details>

The three categories for <i>loan_status</i> are: 
- Fully paid: Applicant has fully paid the loan.
- Current: Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. 
- Charged-off: Applicant has not paid the instalments in due time for a long period of time, i.e. they have <b>defaulted</b> on the loan

### Ignore "Current" value in <i>loan_status</i> column

Filter out loans with "Current" <i>loan_status</i>, since these are not fully paid nor have defaulted.

In [22]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df = loan_df[loan_df['loan_status'] != 'Current']
```

</details>

### Convert <i>loan_status</i> to binary

<i>loan_status</i> is a categorical variable which is not suitable for numerical computations. 
We need to convert it to binary so we can calculate the average default rate and be able to compare it against the independent variables.

Change "Charged Off" to True/1 and "Fully Paid" to False/0. Make sure the data type is either bool or int.

In [23]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

What does <code>loan_df['loan_status'] == "Charged Off"</code> return?

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>


```python
loan_df['loan_status'] = loan_df['loan_status'] == "Charged Off"
```

</details>

# Step 5. EDA
Now that we have high-quality data, we can perform Exploratory Data Analysis. Effective EDA lets us uncover valuable insights by finding patterns in our data, visualizing relationships between its variables, and building intuition about what we're working with. 

## What is the interest rate distribution?

Create a [histogram](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html) for the numerical column <i>int_rate</i>. 

Give an approximate range where most interest rate values fall on.

In [24]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>
Interest rate values are more concentrated between 10% and 15%.

```python
loan_df['int_rate'].hist() 
```
</details>

## Target variable analysis

### What's the distribution of `loan_status`?

Use [value_counts()](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) to get the total row count for both defaulted loans and paid loans, and plot the resulting Series using a [pie plot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.pie.html).

Set the plot's title as "Loan Status Distribution".

Tip: Set the parameter `autopct` on `plot.pie()` to `'%1.1f%%'` to show the percentages on the plot.

In [25]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
pie_plot = loan_df['loan_status'].value_counts().plot.pie(
    title="Loan Status Distribution", 
    autopct='%1.1f%%', 
    labels=['Paid', 'Defaulted'], 
    explode=(0, 0.1))
plt.ylabel('') 
plt.show()     
```
</details>

Note: We can see that the dataset is not balanced with only 14.3% of defaulted loans.

The strategy to identify which features affect loan default the most, is to compare the average default rates across the various independent variables.

The average default rate is
$$ \text{Average Default Rate} = \frac{\text{Number of defaulted loans}}{\text{Total number of loans}} $$

This can be computed as 
```python
(loan_df['loan_status'] == True).sum() / loan_df.shape[0]
```
which is the same as
```python
loan_df['loan_status'].sum() / loan_df.shape[0]
```
which is simply
```python
loan_df['loan_status'].mean()
```

### What is the Average Default Rate by <i>term</i>?

Knowing that Average Default Rate can be computed as <code>loan_df['loan_status'].mean()</code>, calculate this metric for each group of values in column <i>term</i>.

In [26]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html"><code>groupby()</code></a> method. 

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
loan_df.groupby('term')['loan_status'].mean()
```
</details>

This tell us that loans with term 60 months have a higher default rate, i.e., the default rate is increasing with respect to <i>term</i>, hence the chances of a loan getting defaulted is higher for 60 months than 36 months.

Thus, <i>term</i> seems like a good indicator of loan default.

### Create function to calculate the Average Default Rate by any column

Generalise the code above by creating a function called <code>get_default_rate()</code> that takes any categorical column from the dataset and returns the Average Default Rate for each column value.

This will be useful to see the relationship between an independent column and the target variable.

In [27]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
def get_default_rate(column):
    return loan_df.groupby(column)['loan_status'].mean()

# Example
get_default_rate('term')
```
</details>

## Grade analysis

Loan grading is a classification system that involves assigning a quality score to a loan based on risk factors. 'A' grade loans represent the lowest risk while 'G' grade loans are the riskiest.

### *grade* vs *int_rate*

Given *grade*'s definition, we can speculate that riskier grades (higher grades) will have higher interest rates. Let's confirm this by plotting *grade* against <i>int_rate</i>.

For each grade, calculate the average of *int_rate* and plot the result using [`plot.bar()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html). 

In [28]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html"><code>groupby()</code></a> method. 

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>
We can verify that the higher the grade, the higher is the average interest rate.

```python
loan_df.groupby('grade').int_rate.mean().plot.bar()
```
</details>

### What is the *grade*'s distribution?

What grades appear the most and the least in our data? 

In [29]:
# Your code


<details style="font-size: 16px">
<summary>Hint</summary>

Use <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html"><code>value_counts()</code></a> method. 

</details>

<details style="font-size: 16px">
<summary>Sample solution</summary>
Grades 'A' and 'B' have the highest number of loans. This makes sense since they are associated with lower risk loans. 

The company is already lending only a small amount of loans to people classified in higher risk grades, 'F' and 'G'. 

```python
loan_df.grade.value_counts().plot.bar()
```
</details>

### Is *grade* beneficial to predict loan default?

Hypothesis: Loans with higher grade have higher default rate.

Using the previously created function `get_default_rate()`, build a bar plot where the xaxis shows each grade and the yaxis shows the average default rate for each grade.

Set the yaxis label to "Average Default Rate". 

The plot's title should be "Loan Default Rate by grade".

In [30]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>
There's an increasing trend observed for higher grades; Grade G with the highest default rate.

Answer: Grade does seem to affect the default rate since the higher the grade, the higher the percentage of defaulted loans.

```python
get_default_rate('grade').plot(kind='bar')
plt.title("Loan Default Rate by grade")
plt.xlabel("grade")
plt.ylabel("Average Default Rate")
plt.show()
```
</details>

## Create function to calculate the Average Default Rate by any column

Generalise the code above by creating a function called `plot_bar` that plots the Average Default Rate for each grouping of any categorical column passed as a parameter. 

The title should be "Loan Default Rate by \[column\]", where \[column\] is the specific column you are plotting.

In [31]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>

```python
def plot_bar(column):
    get_default_rate(column).plot(kind='bar')
    plt.title("Loan Default Rate by " + column)
    plt.xlabel(column)
    plt.ylabel("Default Rate")
    plt.show()
```
</details>

### Is *int_rate_range* beneficial to predict loan default?

Hypothesis: The higher the interest rate, the higher the default rate. 

In [32]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>
Answer: Interest rate seems a strong predictor of loan default, since we can see that default rate is increasing with the interest rate values.

```python
plot_bar('int_rate_range')
```
</details>

### Is *annual_inc_range* beneficial to predict loan default?

Hypothesis: The lower the annual income, the higher the default rate. 

In [33]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>
Low income earners tend to default more. 

Answer: Annual income seems to affect the default rate, since we can observe a downward trend, which suggests that people with lower income are more likely to default.

```python
plot_bar('annual_inc_range')
```
</details>

### Is *emp_length* beneficial to predict loan default?

In [34]:
# Your code


<details style="font-size: 16px">
<summary>Sample solution</summary>
We can see that default rate is more or less constant, thus loan default does not seem to depend on Employment Length.

```python
plot_bar('emp_length')
```
</details>

# BONUS: Explore the dataset further

Suggestion - Investigate *purpose* column:
- What are the different purpose reasons?
- What is the main purpose for loan defaults?
- Is *purpose* beneficial to predict loan default?

In [35]:
# Your code


<span style="display:none">Rita Vale - rita.martins.vale@gmail.com</span>